In [1]:
import re
import json
import time
import os
import sys
import numpy as np
import qiskit

from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.converters import circuit_to_dag

# Dynamically add the project root to sys.path
project_root = os.path.abspath(os.path.join(os.path.dirname(os.getcwd())))  # Adjust path to root
sys.path.append(project_root)

In [2]:
from src.CE_module import construct_qcc_circuit, CE_recur_tree
from src.CA_module import extract_CNOT_network, update_probabilities, CA_post_QAOA

In [3]:
#QAOA for MAXCUT 
cost_hamiltonian = ['IIIZZ', 'IIZIZ', 'ZIIIZ', 'IIZZI', 'IZZII', 'ZZIII']
mixer_hamiltonian = ['XIIII', 'IXIII', 'IIXII', 'IIIXI', 'IIIIX']
test_paulis = (cost_hamiltonian + mixer_hamiltonian)* 2
test_params = [3.271] * 6 +   [2.874] * 5 + [2.641] * 6 + [0.9339]* 5
start_time = time.time()
opt_qc, append_clifford, sorted_entanglers = CE_recur_tree(entanglers=test_paulis, params=test_params, barrier=False)
opt_qiskit = transpile(opt_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])

end_time = time.time()
combined_time = end_time - start_time


origin_qc = construct_qcc_circuit(entanglers = test_paulis, params = test_params, barrier=False)
#generate the qiskit optimized circuit
origin_qiskit = transpile(origin_qc, optimization_level = 3, basis_gates = ["cx", "sx", "x", "rz"])
origin_qiskit.measure_active() 
origin_qiskit.count_ops()['cx']

24

In [4]:
append_clifford.draw()

┌───┐                     
q_0: ────────────■──┤ H ├─X───■────■────■─────
               ┌─┴─┐├───┤ │   │  ┌─┴─┐  │     
q_1: ───────■──┤ X ├┤ H ├─┼───┼──┤ X ├──┼───X─
          ┌─┴─┐├───┤└───┘ │   │  └───┘┌─┴─┐ │ 
q_2: ──■──┤ X ├┤ H ├──────┼───┼───────┤ X ├─X─
       │  ├───┤└───┘      │   │       └───┘   
q_3: ──┼──┤ H ├───────────┼───┼───────────────
     ┌─┴─┐├───┤           │ ┌─┴─┐             
q_4: ┤ X ├┤ H ├───────────X─┤ X ├─────────────
     └───┘└───┘             └───┘

In [5]:
opt_qc.count_ops()

OrderedDict([('cx', 22), ('rz', 22), ('h', 15)])

In [6]:
opt_qc.draw()

┌───┐    »
q_0: ──■─────────────────■─────────────────■─────────────────■──────┤ H ├────»
     ┌─┴─┐┌───────────┐  │                 │      ┌───┐      │  ┌───┴───┴───┐»
q_1: ┤ X ├┤ Rz(3.271) ├──┼─────────────────┼──────┤ X ├──────┼──┤ Rz(3.271) ├»
     └───┘└───────────┘┌─┴─┐┌───────────┐  │      └─┬─┘      │  └───┬───┬───┘»
q_2: ──────────────────┤ X ├┤ Rz(3.271) ├──┼────────■────────┼──────┤ X ├────»
                       └───┘└───────────┘  │               ┌─┴─┐    └─┬─┘    »
q_3: ──────────────────────────────────────┼───────────────┤ X ├──────■──────»
                                         ┌─┴─┐┌───────────┐└───┘             »
q_4: ────────────────────────────────────┤ X ├┤ Rz(3.271) ├──────────────────»
                                         └───┘└───────────┘                  »
«                                        ┌───┐    ┌───────────┐┌───┐»
«q_0: ───────────────────────────────────┤ X ├────┤ Rz(2.874) ├┤ H ├»
«         ┌───┐    ┌───────────┐┌───┐    └─┬─┘    ├───────────┤├───┤»
«q_1: ────┤ H ├────┤ Rz(2.874) ├┤ X ├──────┼──────┤ Rz(2.874) ├┤ H ├»
«     ┌───┴───┴───┐└───┬───┬───┘└─┬─┘      │      └───────────┘├───┤»
«q_2: ┤ Rz(3.271) ├────┤ H ├──────■────────┼────────────■──────┤ H ├»
«     └───────────┘    ├───┤               │          ┌─┴─┐    ├───┤»
«q_3: ──────■──────────┤ H ├───────────────■──────────┤ X ├────┤ X ├»
«         ┌─┴─┐    ┌───┴───┴───┐┌───┐┌───────────┐    └───┘    └─┬─┘»
«q_4: ────┤ X ├────┤ Rz(3.271) ├┤ H ├┤ Rz(2.874) ├───────────────■──»
«         └───┘    └───────────┘└───┘└───────────┘                  »
«                                    ┌───┐┌───────────┐                  »
«q_0: ───────────────■───────────────┤ X ├┤ Rz(2.641) ├───────────────■──»
«     ┌───────────┐  │               └─┬─┘└───┬───┬───┘┌───────────┐  │  »
«q_1: ┤ Rz(2.641) ├──┼─────────────────┼──────┤ X ├────┤ Rz(2.641) ├──┼──»
«     └───────────┘┌─┴─┐┌───────────┐  │      └─┬─┘    └───┬───┬───┘  │  »
«q_2: ─────────────┤ X ├┤ Rz(2.641) ├──┼────────■──────────┤ H ├──────┼──»
«     ┌───────────┐├───┤└───────────┘  │                   ├───┤      │  »
«q_3: ┤ Rz(2.874) ├┤ H ├───────────────┼────────■──────────┤ H ├──────┼──»
«     └───┬───┬───┘└───┘               │      ┌─┴─┐    ┌───┴───┴───┐┌─┴─┐»
«q_4: ────┤ H ├────────────────────────■──────┤ X ├────┤ Rz(2.641) ├┤ X ├»
«         └───┘                               └───┘    └───────────┘└───┘»
«         ┌───┐     ┌────────────┐                                          »
«q_0: ────┤ H ├─────┤ Rz(0.9339) ├──────────────────────────────────────────»
«         └───┘     └────────────┘    ┌───┐                            ┌───┐»
«q_1: ────────────────────■───────────┤ H ├────────────────────────────┤ X ├»
«     ┌────────────┐      │           └───┘                            └─┬─┘»
«q_2: ┤ Rz(0.9339) ├──────┼──────────────────────────────────────────────┼──»
«     └────────────┘      │                         ┌───┐┌────────────┐  │  »
«q_3: ────────────────────┼─────────────────────────┤ X ├┤ Rz(0.9339) ├──┼──»
«                       ┌─┴─┐     ┌───────────┐┌───┐└─┬─┘└────────────┘  │  »
«q_4: ──────────────────┤ X ├─────┤ Rz(2.641) ├┤ H ├──■──────────────────■──»
«                       └───┘     └───────────┘└───┘                        »
«                                                
«q_0: ────────────────■──────────────────────────
«     ┌────────────┐┌─┴─┐                        
«q_1: ┤ Rz(0.9339) ├┤ X ├──■─────────────────────
«     └────────────┘└───┘┌─┴─┐                   
«q_2: ───────────────────┤ X ├──■────────────────
«                        └───┘  │                
«q_3: ──────────────────────────┼────────────────
«                             ┌─┴─┐┌────────────┐
«q_4: ────────────────────────┤ X ├┤ Rz(0.9339) ├
«                             └───┘└────────────┘

In [7]:
append_clifford.draw()

┌───┐                     
q_0: ────────────■──┤ H ├─X───■────■────■─────
               ┌─┴─┐├───┤ │   │  ┌─┴─┐  │     
q_1: ───────■──┤ X ├┤ H ├─┼───┼──┤ X ├──┼───X─
          ┌─┴─┐├───┤└───┘ │   │  └───┘┌─┴─┐ │ 
q_2: ──■──┤ X ├┤ H ├──────┼───┼───────┤ X ├─X─
       │  ├───┤└───┘      │   │       └───┘   
q_3: ──┼──┤ H ├───────────┼───┼───────────────
     ┌─┴─┐├───┤           │ ┌─┴─┐             
q_4: ┤ X ├┤ H ├───────────X─┤ X ├─────────────
     └───┘└───┘             └───┘

In [8]:
cnot_network, hadamard_counts = extract_CNOT_network(append_clifford)

In [9]:
opt_qc = CA_post_QAOA(opt_qc, hadamard_counts)

In [10]:
#Run the original circuit on a noiseless simulator:
simulator = AerSimulator()
origin_result = simulator.run(origin_qiskit, shots = 10000).result()

In [11]:
def get_top_k_values(input_dict, k):
    # Sort the dictionary by values in descending order and return the top k items
    sorted_items = sorted(input_dict.items(), key=lambda item: item[1], reverse=True)
    return sorted_items[:k]

In [12]:
origin_probs= origin_result.get_counts()

In [13]:
top_k_states_origin = get_top_k_values(origin_probs, 20)
print(top_k_states_origin)

[('11111', 3637), ('11011', 1128), ('11110', 1019), ('11101', 714), ('01111', 655), ('10111', 636), ('11010', 382), ('01011', 188), ('10110', 186), ('11100', 176), ('00111', 168), ('10011', 155), ('11001', 152), ('01110', 141), ('10101', 131), ('01101', 128), ('10010', 57), ('01010', 45), ('00011', 43), ('11000', 42)]


In [14]:
#Run the optimized circuit on a noiseless simulator:

opt_result = simulator.run(opt_qc, shots = 10000).result()

In [15]:
dag = circuit_to_dag(cnot_network)
updated_states = {}

updated_probs = update_probabilities(opt_result.get_counts(), dag)


In [16]:
from qiskit.quantum_info.analysis import hellinger_fidelity

In [17]:
hellinger_fidelity(origin_probs, updated_probs)

0.9980825883296066

In [18]:
top_k_states_origin = get_top_k_values(origin_probs, 20)
print(top_k_states_origin)

[('11111', 3637), ('11011', 1128), ('11110', 1019), ('11101', 714), ('01111', 655), ('10111', 636), ('11010', 382), ('01011', 188), ('10110', 186), ('11100', 176), ('00111', 168), ('10011', 155), ('11001', 152), ('01110', 141), ('10101', 131), ('01101', 128), ('10010', 57), ('01010', 45), ('00011', 43), ('11000', 42)]


In [19]:
top_k_states_updated = get_top_k_values(updated_probs, 20)
print(top_k_states_updated)

[('11111', 3580), ('11011', 1074), ('11110', 1024), ('10111', 712), ('11101', 698), ('01111', 660), ('11010', 338), ('01011', 199), ('10110', 197), ('11100', 186), ('01110', 175), ('11001', 169), ('00111', 157), ('10011', 141), ('10101', 131), ('01101', 117), ('10010', 51), ('11000', 48), ('00011', 47), ('01010', 47)]
